In [1]:
import psi4
import numpy as np
from P4toC4_aux import *

In [2]:
def my_basisspec(mol, role):
    #basstrings = {}
    mol.set_basis_all_atoms("cc-pVTZ", role=role)
    mol.set_basis_by_symbol("H", "sto-3g", role=role)
    #return basstrings

psi4.qcdb.libmintsbasisset.basishorde['MYMIX'] = my_basisspec

In [3]:
psi4.set_memory('500 MB')
psi4.core.set_output_file('output.dat', False)
psi4.core.set_global_option("BASIS", "mymix")
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("D_CONVERGENCE", 1e-8)
psi4.core.set_global_option("PUREAM", "True")

# O
# H 1 0.96
# H 1 0.96 2 104.5

h2o = psi4.geometry("""
0 1
O            0.000000000000     0.000000000000    -0.065775570547
H            0.000000000000    -0.759061990794     0.521953018286
H            0.000000000000     0.759061990794     0.521953018286
symmetry c1
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-76.03212183522113

In [4]:
C=np.array(wf.Ca())
eps = np.array(wf.epsilon_a())
naos, nmos = C.shape
mo1,mo2=1,5  # H2O valence
print('   ', end='')
for mo in range(mo1, mo2):
    print(f'  {27.2114*eps[mo]:10.6f}', end='')
print()
for ao in range(naos):
    print(f'{ao:3d}', end='')
    for mo in range(mo1, mo2):
        print(f'  {C[ao,mo]:10.6f}', end='')
    print()

     -36.692685  -19.461198  -15.876963  -13.934455
  0    0.015547   -0.000000    0.004848    0.000000
  1    0.549285   -0.000000   -0.167264    0.000000
  2    0.124236   -0.000000   -0.058009   -0.000000
  3    0.234356   -0.000000   -0.252619   -0.000000
  4    0.038134    0.000000    0.308942   -0.000000
  5   -0.000000    0.000000    0.000000    0.357487
  6    0.000000    0.273744   -0.000000    0.000000
  7    0.090270    0.000000    0.399798   -0.000000
  8   -0.000000    0.000000    0.000000    0.454302
  9    0.000000    0.385401   -0.000000   -0.000000
 10   -0.012726    0.000000    0.306540   -0.000000
 11   -0.000000    0.000000    0.000000    0.409385
 12    0.000000    0.179938    0.000000    0.000000
 13    0.000805    0.000000    0.003046   -0.000000
 14   -0.000000   -0.000000    0.000000    0.002748
 15   -0.000000    0.013912   -0.000000   -0.000000
 16   -0.004078    0.000000   -0.004461   -0.000000
 17    0.000000   -0.000000   -0.000000    0.000000
 18    0.005

In [6]:
# Print a new basis set in GENBAS format
basisset=wf.basisset()
print(basisset.genbas())

O:P4_1
Psi4 basis MY_BASISSPEC for element O atom 1

  4
    0    1    2    3
    4    3    2    1
   10    5    2    1

 15330.0000000  2299.0000000   522.4000000   147.3000000    47.5500000
    16.7600000     6.2070000     0.6882000     1.7520000     0.2384000

 0.0005080 -0.0001150  0.0000000  0.0000000 
 0.0039290 -0.0008950  0.0000000  0.0000000 
 0.0202430 -0.0046360  0.0000000  0.0000000 
 0.0791810 -0.0187240  0.0000000  0.0000000 
 0.2306870 -0.0584630  0.0000000  0.0000000 
 0.4331180 -0.1364630  0.0000000  0.0000000 
 0.3502600 -0.1757400  0.0000000  0.0000000 
-0.0081540  0.6034180  0.0000000  0.0000000 
 0.0000000  0.0000000  1.0000000  0.0000000 
 0.0000000  0.0000000  0.0000000  1.0000000 

    34.4600000     7.7490000     2.2800000     0.7156000     0.2140000

 0.0159280  0.0000000  0.0000000 
 0.0997400  0.0000000  0.0000000 
 0.3104920  0.0000000  0.0000000 
 0.0000000  1.0000000  0.0000000 
 0.0000000  0.0000000  1.0000000 

     2.3140000     0.6450000

 1.0000000  

In [6]:
mapping, scale =basis_mapping(wf.basisset(), verbose=0)
mapping

array([ 0,  1,  2,  7,  3,  5,  8,  4,  6,  9, 11, 13, 12, 10, 14, 15])

In [7]:
# Psi4
C=np.array(wf.Ca())
print('Psi4 MOs:', wf.nmo())
# Cfour
norb, cs = read_oldmos('OLDMOS.H2O.PVDZ', verbose=1)

Psi4 MOs: 16
reading orbitals from OLDMOS.H2O.PVDZ
  144 lines, 4 orbitals at the bottom
  nAOs = nMOs = 24


In [7]:
mo=5
ao1,ao2=3,9  # O-p for PVDZ
ao1,ao2=9,14 # O-d for PVDZ
print('    Cfour        Psi4')
for ao in range(ao1,ao2):
    print(f'{ao:3d} {cs[ao,mo]:10.6f} {C[ao,mo]:10.6f}')

    Cfour        Psi4
  9  -0.002174  -0.007531
 10  -0.000000   0.000000
 11  -0.000000  -0.000000
 12   0.004922   0.009844
 13  -0.000000   0.000000


In [8]:
C2 = psi4_to_c4(C,mapping, scale)

In [9]:
mo=5
ao1,ao2=3,9  # O-p for PVDZ
ao1,ao2=9,14 # O-d for PVDZ
#ao1,ao2=3,14 # O p and d
print('    Cfour        Psi4')
for ao in range(ao1,ao2):
    print(f'{ao:3d} {cs[ao,mo]:10.6f} {C2[ao,mo]:10.6f}')

    Cfour        Psi4
  9  -0.002174  -0.002174
 10  -0.000000   0.000000
 11  -0.000000   0.000000
 12   0.004922   0.004922
 13  -0.000000  -0.000000


In [10]:
np.max(cs-C2)

6.733795379076923e-07

In [11]:
write_oldmos('PSIMOS', C2)